In [3]:
import requests
from bs4 import BeautifulSoup

domain = "https://help.jp.mercari.com"
top_page_url = f"{domain}/guide/"

In [4]:
categories = []
articles = []

def get_articles_and_categories(link: str):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    tags = soup.find_all("a")
    for tag in tags:
        href = tag.get("href")
        if href.startswith(domain):
            relative_href = href[len(domain):]
        else:
            relative_href = href
        print(f"href:{relative_href}")
        if href.startswith("/guide/categories/") and href not in categories:
            categories.append(href)
            get_articles_and_categories(f"{domain}{relative_href}")
        elif href.startswith("/guide/articles/") and href not in articles:
            articles.append(href)

get_articles_and_categories(top_page_url)

href:https://jp.mercari.com/
href:/guide/
href:https://www.mercari.com/jp/guide/beginner/
href:/guide/articles/62/
href:/guide/articles/20/
href:/guide/articles/274/
href:/guide/categories/6/
href:https://jp.mercari.com/
href:/guide/
href:/guide/categories/1418/
href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/23/
href:/guide/articles/361/
href:/guide/articles/928/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.m

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/568/
href:/guide/articles/585/
href:/guide/articles/586/
href:/guide/articles/1160/
href:/guide/articles/1300/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/data_security
href:https://static.jp.mercari.

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/41/
href:/guide/articles/42/
href:/guide/articles/630/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/data_security
href:https://static.jp.mercari.com/tokutei
href:https://static.jp.mercari.com/shikin_ke

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/274/
href:/guide/articles/560/
href:/guide/articles/95/
href:/guide/articles/96/
href:/guide/articles/422/
href:/guide/articles/480/
href:/guide/articles/746/
href:/guide/articles/97/
href:/guide/articles/978/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.merc

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/258/
href:/guide/articles/259/
href:/guide/articles/1128/
href:/guide/articles/260/
href:/guide/articles/261/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/data_security
href:https://static.jp.mercari.c

...

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/576/
href:/guide/articles/561/
href:/guide/articles/1241/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/data_security
href:https://static.jp.mercari.com/tokutei
href:https://static.jp.mercari.com/shikin

href:https://jp.mercari.com/
href:/guide/
href:/guide/categories/1460/
href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/741/
href:/guide/articles/567/
href:/guide/articles/569/
href:/guide/articles/742/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/da

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/1108/
href:/guide/articles/1109/
href:/guide/articles/1110/
href:/guide/articles/1113/
href:/guide/articles/1114/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/data_security
href:https://static.jp.merca

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/993/
href:/guide/articles/996/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https://support.mercari-shops.com/hc/ja/articles/900005891046
href:https://static.jp.mercari.com/compliance
href:https://static.jp.mercari.com/data_security
href:https://static.jp.mercari.com/tokutei
href:https://static.jp.mercari.com/shikin_kessai
href:https://about.

href:https://jp.mercari.com/
href:/guide/
href:/guide/articles/246/
href:/guide/articles/352/
href:/guide/articles/688/
href:/guide/articles/353/
href:/guide/articles/591/
href:/guide/articles/248/
href:/guide/articles/255/
href:/guide/articles/1019/
href:/guide/articles/243/
href:/guide/articles/850/
href:/guide/articles/250/
href:/guide/articles/354/
href:https://about.mercari.com/
href:https://careers.mercari.com/jp/
href:https://about.mercari.com/press/news/
href:https://jp-news.mercari.com/
href:https://about.mercari.com/press/press-kit/mercari/
href:https://mercari.com/
href:https://mercari-shops.com/
href:https://souzoh.com/
href:https://souzoh.com/news
href:/
href:/guide/categories/1404/
href:/guide/categories/3/
href:/guide/categories/12/
href:/guide/categories/542/
href:/guide/categories/4/
href:https://support.mercari-shops.com/hc/ja
href:https://jp.mercari.com/keywords
href:/cases
href:https://static.jp.mercari.com/privacy
href:https://static.jp.mercari.com/tos
href:https:/

In [5]:
articles

['/guide/articles/62/',
 '/guide/articles/20/',
 '/guide/articles/274/',
 '/guide/articles/23/',
 '/guide/articles/361/',
 '/guide/articles/928/',
 '/guide/articles/24/',
 '/guide/articles/25/',
 '/guide/articles/26/',
 '/guide/articles/29/',
 '/guide/articles/30/',
 '/guide/articles/58/',
 '/guide/articles/38/',
 '/guide/articles/32/',
 '/guide/articles/33/',
 '/guide/articles/34/',
 '/guide/articles/35/',
 '/guide/articles/36/',
 '/guide/articles/37/',
 '/guide/articles/826/',
 '/guide/articles/1030/',
 '/guide/articles/43/',
 '/guide/articles/39/',
 '/guide/articles/40/',
 '/guide/articles/684/',
 '/guide/articles/685/',
 '/guide/articles/686/',
 '/guide/articles/692/',
 '/guide/articles/568/',
 '/guide/articles/585/',
 '/guide/articles/586/',
 '/guide/articles/1160/',
 '/guide/articles/1300/',
 '/guide/articles/949/',
 '/guide/articles/951/',
 '/guide/articles/744/',
 '/guide/articles/957/',
 '/guide/articles/952/',
 '/guide/articles/950/',
 '/guide/articles/954/',
 '/guide/article

In [6]:
categories

['/guide/categories/6/',
 '/guide/categories/1418/',
 '/guide/categories/1419/',
 '/guide/categories/1420/',
 '/guide/categories/1421/',
 '/guide/categories/693/',
 '/guide/categories/1447/',
 '/guide/categories/1448/',
 '/guide/categories/1449/',
 '/guide/categories/1450/',
 '/guide/categories/1451/',
 '/guide/categories/1422/',
 '/guide/categories/1404/',
 '/guide/categories/2/',
 '/guide/categories/9/',
 '/guide/categories/795/',
 '/guide/categories/1478/',
 '/guide/categories/800/',
 '/guide/categories/1416/',
 '/guide/categories/1417/',
 '/guide/categories/977/',
 '/guide/categories/1441/',
 '/guide/categories/1442/',
 '/guide/categories/1443/',
 '/guide/categories/1/',
 '/guide/categories/5/',
 '/guide/categories/1406/',
 '/guide/categories/1412/',
 '/guide/categories/1413/',
 '/guide/categories/1414/',
 '/guide/categories/1415/',
 '/guide/categories/715/',
 '/guide/categories/728/',
 '/guide/categories/732/',
 '/guide/categories/735/',
 '/guide/categories/13/',
 '/guide/categori

In [7]:
abs_url = f"{domain}{articles[0]}"
page = requests.get(abs_url)
soup = BeautifulSoup(page.content, "html.parser")
soup

<!DOCTYPE html>
<html lang="ja"><head><meta charset="utf-8"/><meta content="initial-scale=1, viewport-fit=cover" name="viewport"/><title>メルカリ出品までの流れ・売り方 - メルカリ スマホでかんたん フリマアプリ</title><meta content="このガイドでは、出品完了までの流れをご紹介します。出品完了後の商品については、こちらをご覧ください。商品写真を登録する商品の詳細を設定する商品名と説明を入力する配送について設定する販売価格を設定する「出品する」をタッチする1.商品写真を登録するまずは、商品の写真を登録しましょう。写真の登録方法は、アプリ内で撮影する方法とご利用端末のアルバムから選択する方法があります。出品ホーム画面のカメラマークを選択し、写真を登録してください。※撮影する際には、正面・横・裏側・傷がある部分など、詳細を撮影してください詳しい手順は下記動画をご確認ください。▼ 写真を撮って出品する再生されない場合はコチラ▼ アルバムから選んで出品する再生されない場合はコチラ▼ 商品撮影のキホン再生されない場合はコチラ商品の詳細が伝わるように写真を撮影してください。必ず自身で撮影したものを使用してください。Webサイトの場合は、出品したい商品の写真をパソコンに取り込んでから「出品」ボタンをクリックしてください。バーコード出品については、 「バーコード出品」機能についてをご参照ください。写真は10枚まで登録することが可能です。▲トップへ戻る2.商品の詳細を設定するカテゴリーと商品の状態について設定しましょう。■ カテゴリー出品する商品のカテゴリーを選択すると、サイズやブランドなどの選択肢が追加されます。なお、追加した写真の情報をもとに自動でカテゴリーが設定されることがあります。設定されたカテゴリーに商品が属さない場合は、改めて選択し直してください。※カテゴリーを選び直すと、サイズやブランドなど任意で入力した情報もリセットされます■ 商品の状態出品する商品の使用状態を5つのレベルから設定しましょう。新品、未使用：購入してからあまり時間が経っておらず、一度も使用していない未使用に近い：数回しか使用しておらず、傷や汚れがな

In [8]:
soup.find("article")

<article><header><h3 class="text-15 line-[1.1] pb-ds-16 pc:b-ds-24 tw-border border-b border-ds-primary">メルカリ出品までの流れ・売り方</h3></header><div class="pt-ds-16 pb-ds-24 pc:pt-ds-24 pc:pb-ds-40"><p class="first:m-0 my-ds-16">このガイドでは、出品完了までの流れをご紹介します。<br/>
<strong>出品完了後の商品</strong>については、<span class="merTextLink"><a class="no-underline hover:underline break-all" data-name="mg:article-link" href="/guide/articles/62/#0">こちら</a></span>をご覧ください。</p>
<div style="border:3px solid #F2F2F2;padding:10px">
<ol class="mt-ds-16 list-decimal pl-ds-24"><li class="mt-ds-8"><span class="merTextLink"><a class="no-underline hover:underline break-all" data-name="mg:article-link" href="/guide/articles/62/#1">商品写真を登録する</a></span></li>
<li class="mt-ds-8"><span class="merTextLink"><a class="no-underline hover:underline break-all" data-name="mg:article-link" href="/guide/articles/62/#2">商品の詳細を設定する</a></span></li>
<li class="mt-ds-8"><span class="merTextLink"><a class="no-underline hover:underline break-all" data-name

In [9]:
soup.find("article").get_text(strip=True)

'メルカリ出品までの流れ・売り方このガイドでは、出品完了までの流れをご紹介します。出品完了後の商品については、こちらをご覧ください。商品写真を登録する商品の詳細を設定する商品名と説明を入力する配送について設定する販売価格を設定する「出品する」をタッチする1.商品写真を登録するまずは、商品の写真を登録しましょう。写真の登録方法は、アプリ内で撮影する方法とご利用端末のアルバムから選択する方法があります。出品ホーム画面のカメラマークを選択し、写真を登録してください。※撮影する際には、正面・横・裏側・傷がある部分など、詳細を撮影してください詳しい手順は下記動画をご確認ください。▼ 写真を撮って出品する再生されない場合はコチラ▼ アルバムから選んで出品する再生されない場合はコチラ▼ 商品撮影のキホン再生されない場合はコチラ商品の詳細が伝わるように写真を撮影してください。必ず自身で撮影したものを使用してください。Webサイトの場合は、出品したい商品の写真をパソコンに取り込んでから「出品」ボタンをクリックしてください。バーコード出品については、「バーコード出品」機能についてをご参照ください。写真は10枚まで登録することが可能です。▲トップへ戻る2.商品の詳細を設定するカテゴリーと商品の状態について設定しましょう。■ カテゴリー出品する商品のカテゴリーを選択すると、サイズやブランドなどの選択肢が追加されます。なお、追加した写真の情報をもとに自動でカテゴリーが設定されることがあります。設定されたカテゴリーに商品が属さない場合は、改めて選択し直してください。※カテゴリーを選び直すと、サイズやブランドなど任意で入力した情報もリセットされます■ 商品の状態出品する商品の使用状態を5つのレベルから設定しましょう。新品、未使用：購入してからあまり時間が経っておらず、一度も使用していない未使用に近い：数回しか使用しておらず、傷や汚れがない目立った傷や汚れなし：よく見ないとわからない程度の傷や汚れがあるやや傷や汚れあり：中古品とわかる程度の傷や汚れがある傷や汚れあり：誰がみてもわかるような大きな傷や汚れがある全体的に状態が悪い：商品の全体に目立つ傷や汚れ、ダメージがある設定した「商品の状態」は、あくまでも「目安」です。傷や汚れがある商品を出品する場合は、該当箇所を撮影し出品画像に掲載したり、商品説

In [10]:
from llama_index import download_loader

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()
documents = loader.load_data(urls=[f"{domain}{articles[0]}"])

In [11]:
documents

[Document(text='メルカリ出品までの流れ・売り方 - メルカリ スマホでかんたん フリマアプリヘッダーパンくずリストメルカリガイド検索メルカリガイドサイドバーメインコンテンツメルカリ出品までの流れ・売り方このガイドでは、出品完了までの流れをご紹介します。\n出品完了後の商品については、こちらをご覧ください。\n\n商品写真を登録する\n商品の詳細を設定する\n商品名と説明を入力する\n配送について設定する\n販売価格を設定する\n「出品する」をタッチする\n\n\n\n\n1.商品写真を登録する\n\nまずは、商品の写真を登録しましょう。\n写真の登録方法は、アプリ内で撮影する方法とご利用端末のアルバムから選択する方法があります。\n出品ホーム画面のカメラマークを選択し、写真を登録してください。\n※撮影する際には、正面・横・裏側・傷がある部分など、詳細を撮影してください\n詳しい手順は下記動画をご確認ください。\n\n▼ 写真を撮って出品する\n\n再生されない場合はコチラ\n▼ アルバムから選んで出品する\n\n再生されない場合はコチラ\n▼ 商品撮影のキホン\n\n再生されない場合はコチラ\n\n\n商品の詳細が伝わるように写真を撮影してください。\n必ず自身で撮影したものを使用してください。\nWebサイトの場合は、出品したい商品の写真をパソコンに取り込んでから「出品」ボタンをクリックしてください。\nバーコード出品については、 「バーコード出品」機能についてをご参照ください。\n写真は10枚まで登録することが可能です。\n\n\n\n▲トップへ戻る\n\n2.商品の詳細を設定する\n\nカテゴリーと商品の状態について設定しましょう。\n■ カテゴリー\n出品する商品のカテゴリーを選択すると、サイズやブランドなどの選択肢が追加されます。\nなお、追加した写真の情報をもとに自動でカテゴリーが設定されることがあります。設定されたカテゴリーに商品が属さない場合は、改めて選択し直してください。\n※カテゴリーを選び直すと、サイズやブランドなど任意で入力した情報もリセットされます\n\n■ 商品の状態\n出品する商品の使用状態を5つのレベルから設定しましょう。\n\n新品、未使用：購入してからあまり時間が経っておらず、一度も使用していない\n未使用に近い

In [12]:
from llama_index import GPTSimpleVectorIndex, PromptHelper

index = GPTSimpleVectorIndex.from_documents(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 5646 tokens


In [13]:
index

In [14]:
index.query("What language is on this website?")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2006 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens


Response(response='\nJapanese.', source_nodes=[NodeWithScore(node=Node(text='「出品する」をタッチする\nすべての項目の入力・設定が完了したら「出品する」をタッチし、出品は完了です。\n「出品する」を選択できない場合は、必須項目が未入力の可能性がございますので、各項目に抜け漏れがないか再度ご確認ください。\n\n▲トップへ戻る\n\n出品完了後の流れについて\n出品完了後、価格や商品写真・商品情報の変更、商品の取り下げをする場合、購入される前に限り対応が可能です。\n「マイページ＞出品した商品」から希望の商品を選択し、画面下部の「商品の編集」からご対応ください。\n※取引開始後は、価格や商品情報の変更はできませんので、お気をつけください\n■ よくある質問\n取り置き・専用出品の相談をされた\n応じていただいて問題ございませんが、注意点がございます。詳細はこちらをご参照ください。\n値下げ交渉に応じたい\n価格の変更方法については、こちらをご参照ください。\n出品した商品が検索結果/新着に表示されません\n表示されるまでお時間がかかる場合がございます。詳細はこちらをご参照ください。\n商品が売れた！これからの流れは？\n商品を発送し取引完了となりましたら、売上金をご確認ください。詳細はこちらをご参照ください。\n\n\n▲トップへ戻る\n\nメルカリ教室では、「出品の流れ」「梱包・発送方法」「売れるコツ」などを無料で学ぶことができる講座を毎日開催しています。ぜひ、お気軽にご参加ください。解決しない場合お客さまの状況にあわせたご案内を行うため、お問い合わせにはログインが必要です。アカウントをお持ちでない方は次のページから登録をお願いいたします。お問い合わせはこちらログインできない方はこちらフッターメルカリについて会社概要（運営会社）採用情報プレスリリース公式ブログプレスキットメルカリUSメルカリShopsメルカリShops会社概要（運営会社）メルカリShopsプレスリリースヘルプセンターヘルプセンター出品・購入方法配送方法取引開始後のFAQメルペイアカウントメルカリShops出品者向けガイドフリーワードからさがすお問い合わせ一覧プライバシーと利用規約プライバシーポリシーメルカリ利用規約メルカリ

In [15]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
import pandas as pd
import tiktoken
import openai
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity

In [16]:
abs_articles = [domain + article for article in articles]
print(abs_articles)

['https://help.jp.mercari.com/guide/articles/62/', 'https://help.jp.mercari.com/guide/articles/20/', 'https://help.jp.mercari.com/guide/articles/274/', 'https://help.jp.mercari.com/guide/articles/23/', 'https://help.jp.mercari.com/guide/articles/361/', 'https://help.jp.mercari.com/guide/articles/928/', 'https://help.jp.mercari.com/guide/articles/24/', 'https://help.jp.mercari.com/guide/articles/25/', 'https://help.jp.mercari.com/guide/articles/26/', 'https://help.jp.mercari.com/guide/articles/29/', 'https://help.jp.mercari.com/guide/articles/30/', 'https://help.jp.mercari.com/guide/articles/58/', 'https://help.jp.mercari.com/guide/articles/38/', 'https://help.jp.mercari.com/guide/articles/32/', 'https://help.jp.mercari.com/guide/articles/33/', 'https://help.jp.mercari.com/guide/articles/34/', 'https://help.jp.mercari.com/guide/articles/35/', 'https://help.jp.mercari.com/guide/articles/36/', 'https://help.jp.mercari.com/guide/articles/37/', 'https://help.jp.mercari.com/guide/articles/82

In [17]:
from langchain.chat_models import ChatOpenAI
from llama_index import LLMPredictor,ServiceContext

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()
service_context = ServiceContext.from_defaults(llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo")))

documents = loader.load_data(urls=abs_articles)
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
index.save_to_disk("mercari_guide.json")


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1098866 tokens


In [18]:
def response(message):
    qry = f"""
あなたはメルカリのカスタマーサポートです。与えられたインデックスに関連するお客様の質問の対して、丁寧かつ分かりやすく回答します。```で囲まれた質問に対して、質問者の言語で回答します。必要に応じて、動画やスクリーンキャプチャ画像も追加して回答します。

質問: ```{message}```
回答:
    """
    print(index.query(qry))

In [19]:
response("出品方法を教えてください。")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1865 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 216 tokens


出品方法については、以下の手順で行うことができます。
1. メルカリアプリを開きます。
2. 右下にある「出品する」ボタンをタップします。
3. 出品する商品の写真を撮影します。
4. 商品のタイトル、説明、価格などの情報を入力します。
5. 発送方法や配送料金などを設定します。
6. 出品内容を確認し、「出品する」ボタンをタップします。
以上が、出品方法の一般的な手順です。詳しい情報については、ヘルプセンターをご確認ください。


In [20]:
response("公式サイトのURLを教えてください")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1596 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 215 tokens


https://www.mercari.com/ または https://help.mercari.com/ をご覧ください。


In [21]:
response("写真を撮って出品する方法の動画はどこにありますか？")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1759 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 235 tokens


写真を撮って出品する方法の動画は、以下のリンクからご覧いただけます。
【Zoom版】メルカリで売ってみよう
【YouTube版】メルカリで売ってみよう
また、より詳しい情報が必要な場合は、ヘルプセンターから「写真の撮り方」を検索していただくと、より具体的な手順が確認できます。


In [22]:
response("写真を撮って出品する方法の動画のリンクを教えてください。")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1692 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 237 tokens


こちらにメルカリ教室での写真撮影方法の動画がございます。ご参考になさってください。
【Zoom版】メルカリで売ってみよう
【YouTube版】メルカリで売ってみよう
【YouTube版】マネー講座


In [23]:
response("メルカードを発行後支払い方法がメルカードになってしまい以前のスマート払いにしたいのでやり方を教えてください。")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1820 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 265 tokens


お問い合わせありがとうございます。申し訳ありませんが、メルカードの支払い方法を変更することはできません。メルカードの利用停止を行い、スマート払いに切り替えることをお勧めします。詳細については、こちらのガイドをご覧ください。 https://help.jp.mercari.com/guide/articles/1195/


In [25]:
response("メルカード決済ではなく以前のメルペイスマート払いは使えないのですか？")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 3058 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 242 tokens


申し訳ありませんが、現在はメルペイスマートマネーをご利用いただくことができますが、以前のメルペイスマート払いはご利用いただけません。ご了承ください。


In [26]:
response("メルカリポイントとメルカードの2つのみ使用しましたが、このメルカードでの支払いのことを、後払いと仰せなのでしょうか。")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1833 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 266 tokens


はい、メルカードを含む「メルペイのあと払い」でのお支払いは後払いになります。ご利用いただきありがとうございます。


In [28]:
response("""「【メルペイ】3月請求の支払い期限は2023年3月31日(金)です」というタイトルのメールが配信されました。「あと払い2月利用分の支払い期限は2023年3月31日(金)です」とのことですが、メルカリ内での購入において、後払いを利用した記憶がありません。ついては以下2点にご教示願います。
1，メルカリポイントとメルカードの2つのみ使用しましたが、このメルカードでの支払いのことを、後払いと仰せなのでしょうか。
2，この後、どこかへ振込に行くなど、私のほうで必要な対処はありますか
""")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 6119 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 516 tokens


いつもメルカリをご利用いただきありがとうございます。ご質問いただいた件について回答いたします。

1. メルカードでの支払いも、メルペイのあと払いとしてご利用いただけます。そのため、ご利用いただいたメルカードでの支払いが、後払いであった可能性がございます。

2. メールに記載の支払い期限までに、既に支払いが完了している場合は特にご対処は必要ありません。支払いがまだ完了していない場合は、お手数ですがメルペイのあと払いの支払い手続きを行っていただく必要がございます。コンビニ/ATM、もしくはメルペイ残高でお支払いいただけます。また、支払期限を過ぎると延滞事務手数料が加算されることがありますので、お早めにご対応いただけますようお願い申し上げます。

以上、ご確認いただけますようお願い申し上げます。何かご不明な点がございましたら、いつでもお問い合わせください。


In [29]:
response("この品をメルカードでの支払いで購入したのですが、スマートメル払いの支払い方法を設定せよと連絡が来ています。何故カード決済にならないのでしょうか？何か騙されている感じです。どうしたら良いか連絡ください。")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2148 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 336 tokens


お客様、この度はメルカリをご利用いただきありがとうございます。お客様がメルカードを使用して商品を購入し、スマートメル払いの支払い方法を設定するように連絡が来たことについて、大変ご不便をおかけしております。ただし、お客様がご利用されたメルカードは、クレジットカードではなく、前払い式のカードです。このため、商品の支払いにはスマートメル払いの支払い方法を設定する必要があります。お客様がスマートメル払いの設定方法についてわからない場合は、お手数ですがメルカリのヘルプセンターにお問い合わせいただくか、以下のリンクをご参照ください。https://help.mercari.com/jp/sellers/payment/smart-payment/
何かご不明な点がございましたら、お気軽にお問い合わせください。


In [30]:
q = """
ポイント欲しさにメルカードを申し込みましたが、また更にポイントをもらうため、メルペイの後払いをするには何をすればいいですか？コンビニなどで普通にID払いで買い物すればいいのでしょうか？メルカードは利子がつくようですが、それだったら普通にメルペイで買い物した方が良いのでは？メルカードを利用する価値がよくわかりません。高額な買い物ができるということなのでしょうか？メルペイの後払いとメルカードの違いがよくわからないです。あとバーチャルカードもありますよね？バーチャルカードとメルカードが別々の意味があるのでしょうか？
"""
response(q)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2587 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 535 tokens


メルカードを保有している場合、メルカードを含む「メルペイのあと払い」でのお支払いでポイントが還元されます。具体的には、メルカード、iD決済、コード決済、ネット決済、バーチャルカード、およびメルカリでの購入が対象です。普通にID払いで買い物をすることもできますが、ポイント還元の対象外となります。メルカードは利子がつくカードですが、ポイント還元の対象となるため、メルペイの後払いで利用することをおすすめします。メルカードは高額な買い物をする際に便利ですが、ポイント還元のために普段のお買い物でも活用することができます。メルペイの後払いとメルカードの違いは、後払いは購入後に支払いができる方法であり、メルカードはクレジットカードのように使用できるカードです。バーチャルカードは、インターネット上でのショッピングなどに利用できる仮想のカードです。メルカードとバーチャルカードは別々の意味があります。ご不明な点があれば、お気軽にお問い合わせください。


In [31]:
q = """
メルカードの申し込みをしたのですが、後払いや定額払いの設定はするつもりがなく、申し込み時にも「後払い、定額払いにはなりません」と書かれていたように思うのですが、気づいたら「後払い、定額払い」の項目がでるようになっていました。すでに手数料等がかかっているのでしょうか？
"""
response(q)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2426 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 379 tokens


申し込み時に「後払い、定額払いにはなりません」という記載があった場合でも、後払い、定額払いの項目が表示される場合があります。手数料等がかかっている場合は、詳細な状況に応じてご案内する必要があります。詳細はお客様のアカウント情報を確認する必要がありますので、お手数ですがお問い合わせの際はログインいただくか、アカウントをお持ちでない場合はアカウント登録をお願いいたします。


In [32]:
q = """
聞いた事もない「あと払い」という払い方は通常のクレジット払いの事と同じか？メルカードを通常のクレジットカードだと思い申込みしたが、後払いで手数料かかるなら要らない。手数料かからない払い方はあるのか？
"""
response(q)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 6119 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 328 tokens


「後払い」とは、商品やサービスを購入した後に支払いをする方法のことです。クレジットカード払いとは異なり、商品やサービスを購入した後に請求書が送られ、その後に支払いをする方法です。メルカードはクレジットカードではなく、後払い決済サービスを提供しているものです。後払い決済サービスを利用する際には、支払い方法をご自身で選択することができます。一括払いや分割払い、リボ払い、ボーナス払いなどの支払い方法があります。支払い方法によって手数料がかかる場合がありますが、一括払いの場合は一般的に手数料はかかりません。ただし、期限までに支払いが完了しなかった場合には、未払いの利用料金と支払い完了までの期間に応じて遅延損害金が請求される場合があります。詳細は利用するサービス事業者によって異なるので、事前に確認することをおすすめします。


In [33]:
response("""
メルペイスマート払いだとメルカリポイントが1％〜4％つくと見ましたが、メルペイスマート払いをして残高で清算してもポイントはつくのでしょうか？
残高でそのまま購入するとメルカリポイントはつかないので、ポイントを沢山つけたい場合はメルペイスマート払いをしてから残高で精算したほうが良いということでしょうか？
""")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2777 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 407 tokens


はい、メルペイスマート払いで残高で清算してもポイントはつきます。また、残高で直接購入する場合はポイントはつきませんが、メルペイスマート払いをしてから残高で精算することでポイントを獲得することができます。ポイントをたくさん獲得したい場合はこの方法がおすすめです。詳細については、お問い合わせフォームよりお問い合わせください。


In [34]:
response("""商品を出品して売れたお金(残高)で購入したらポイントはつくんでしょうか？ 
売れたお金(残高)で購入するより、残高を口座に振替して、メルカード払いで購入した方がお得なんでしょうか？
""")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2728 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 337 tokens


売上金から購入するポイント（有償ポイント）をおくることができますが、売上金から購入した場合でもポイントは付与されません。また、口座に振り替えた後、メルカード払いで購入した場合でもポイントは付与されません。ただし、口座に振り替えた場合は、振り替え手数料がかかる点にご注意ください。詳細については、メルカリの公式ヘルプページをご覧ください。


In [35]:
response("""メルカードは、紐付けした後に普通に売上金で購入してもポイントは付かないですか？
ポイントが付くのは後払い、定額払いなど限定ですか？
では後払いや定額払い以外で、普段売り上げ金で購入している者にはあまり有効なカードではないという見解で宜しいでしょうか？普段のメルカリの買い物を売り上げ金での購入だと1%〜4%のポイントが付かないという事ですよね？
""")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2163 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 448 tokens


メルカードを紐付けた後、普通に売上金で購入した場合はポイントは付きません。ポイントが付くのはメルペイの後払いや定額払いなど限定された支払い方法になります。普段売上金で購入している方にとっては、ポイント還元の恩恵を受けられないことになりますが、ご利用いただける支払い方法もございますので、ご確認ください。


In [36]:
response("""定額払いを解約したいのですが、どこを探しても解約ページが見つかりません。
""")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2152 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 252 tokens


「定額払いを解約したい場合は？」というページにアクセスして、手続きの詳細をご確認ください。手続きの流れに従って、全ての利用分を支払い完了後に、解約理由を記載のうえで事務局へお問い合わせください。お手続きにはログインが必要です。ログインできない場合は、別のブラウザを使用するか、お問い合わせフォームからお問い合わせください。詳細はこちらのページをご確認ください。```(https://help.jp.mercari.com/guide/articles/956/)```
